In [ ]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-akhrw1k7
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-akhrw1k7
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.1 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=7c2e9a98117d451cfc275fe6d790e869edd51ca7a62d3461ca5dd2dcc318161f
  Stored in directory: /tmp/pip-ephem-wheel-cache-j5wwem37/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
import clip
from torchvision import models

In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/img_align_celeba.zip'  # Adjust path as needed
extract_path = '/content/data/'  # Path to extract files to

# Create the extract directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f'Files extracted to {extract_path}')


Files extracted to /content/data/


In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 16
EPOCHS = 1
LEARNING_RATE = 1e-4

# Load CLIP model
clip_model, preprocess = clip.load("ViT-B/32", device=DEVICE)
clip_model.eval()

# Load ProGAN from PyTorch Hub
progan = torch.hub.load('facebookresearch/pytorch_GAN_zoo:hub', 'PGAN', model_name='celebAHQ-256', pretrained=True, useGPU=torch.cuda.is_available())


100%|███████████████████████████████████████| 338M/338M [00:06<00:00, 55.7MiB/s]
Downloading: "https://github.com/facebookresearch/pytorch_GAN_zoo/zipball/hub" to /root/.cache/torch/hub/hub.zip
Downloading: "https://dl.fbaipublicfiles.com/gan_zoo/PGAN/celebaHQ_s6_i80000-6196db68.pth" to /root/.cache/torch/hub/checkpoints/celebaHQ_s6_i80000-6196db68.pth
100%|██████████| 264M/264M [00:02<00:00, 115MB/s]


Average network found !


In [ ]:
import torch.nn.functional as F

In [ ]:
class MappingNetwork(nn.Module):
    def __init__(self, input_dim=512, latent_dim=512):
        super(MappingNetwork, self).__init__()
        layers = []
        for _ in range(12):
            layers.append(nn.Linear(input_dim, input_dim))
            layers.append(nn.LeakyReLU(0.2))
        layers.append(nn.Linear(input_dim, latent_dim))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [ ]:
import gdown

In [ ]:
# from torchvision import datasets, transforms
# from torch.utils.data import DataLoader

# # Define any transformations you want to apply to your images
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
# ])

# # Use ImageFolder if you have a directory structure like:
# # /content/data/train/class1
# # /content/data/train/class2
# # where each folder represents a class
# dataset = datasets.ImageFolder(root='/content/data/img_align_celeba', transform=transform)

# # Create the DataLoader
# batch_size = 16
# celeba_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# # Check the first batch
# data_iter = iter(celeba_loader)
# images, labels = data_iter.next()
# print(images.shape)  # Check the shape of the images
# print(labels.shape)  # Check the shape of the labels


FileNotFoundError: Couldn't find any class folder in /content/data/img_align_celeba.

In [ ]:
mapping_network = MappingNetwork().to(DEVICE)




# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
# ])
# celeba_dataset = datasets.CelebA(root='./data/img_align_celeba', split='train', transform=transform, download=False)
# celeba_loader = torch.utils.data.DataLoader(celeba_dataset, batch_size=BATCH_SIZE, shuffle=True)



In [ ]:

from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

class RawImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        """
        Args:
            image_dir (str): Directory with raw images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.image_dir = image_dir
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith(('jpg', 'jpeg', 'png'))]  # Filter image files
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_name = self.image_files[idx]
        image_path = os.path.join(self.image_dir, image_name)

        # Open the image
        image = Image.open(image_path).convert("RGB")  # Ensure 3-channel RGB

        # Apply transformations if defined
        if self.transform:
            image = self.transform(image)

        return image

# Define transformations (optional)
# Define any transformations you want to apply to your images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])


# Initialize the dataset
dataset = RawImageDataset(image_dir='/content/data/img_align_celeba', transform=transform)

# Create a DataLoader
batch_size = 16
celeba_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# # Verify the first batch of data
# data_iter = iter(dataloader)
# images = data_iter.next()
# print(images.shape)  # Should print (batch_size, 3, 128, 128) if using the transformations above


In [ ]:
IMAGE_SIZE = 224
discriminator = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
    nn.LeakyReLU(0.2),
    nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
    nn.LeakyReLU(0.2),
    nn.Flatten(),
    nn.Linear(128 * (IMAGE_SIZE // 4) ** 2, 1)
).to(DEVICE)

In [ ]:
# prompt: write code to check if there is a .pth file available for discriminator as well as mapping network if yes load the weights to it and use it in further part of the code

import os

# Check for discriminator .pth file
discriminator_path = 'discriminator.pth'  # Replace with your actual path
if os.path.exists(discriminator_path):
    print("Discriminator weights found. Loading...")
    discriminator.load_state_dict(torch.load(discriminator_path, map_location=DEVICE))
    discriminator.eval()  # Set to evaluation mode
else:
    print("Discriminator weights not found.")

# Check for mapping network .pth file
mapping_network_path = 'mapping_network_progan.pth'  # Replace with your actual path
if os.path.exists(mapping_network_path):
    print("Mapping Network weights found. Loading...")
    mapping_network.load_state_dict(torch.load(mapping_network_path, map_location=DEVICE))
    mapping_network.eval()  # Set to evaluation mode
else:
    print("Mapping Network weights not found.")

Discriminator weights found. Loading...
Mapping Network weights found. Loading...


<ipython-input-12-c155e49d95d6>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  discriminator.load_state_dict(torch.load(discriminator_path, map_location=DEVICE))
<ipython-

In [ ]:

criterion_reconstruction = nn.MSELoss()
criterion_identity = nn.CosineSimilarity(dim=1)
criterion_adversarial = nn.BCEWithLogitsLoss()

# Define a simple discriminator for adversarial loss


# Perceptual Loss Model (LPIPS)
perceptual_model = models.alexnet(pretrained=True).features.to(DEVICE).eval()

# Diversity Loss
def diversity_loss(feature_1, feature_2, img_1, img_2):
    # Flatten images to match feature-like structure
    img_1_flat = img_1.view(img_1.size(0), -1)  # [B, C*H*W]
    img_2_flat = img_2.view(img_2.size(0), -1)  # [B, C*H*W]

    # Compute L1 norms
    dist_features = torch.norm(feature_1 - feature_2, p=1, dim=1)  # [B]
    dist_images = torch.norm(img_1_flat - img_2_flat, p=1, dim=1)  # [B]

    # Compute mean diversity loss
    return torch.mean(dist_features / (dist_images + 1e-8))  # Avoid division by zero


# Optimizers
optimizer = optim.Adam(mapping_network.parameters(), lr=LEARNING_RATE)
discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=LEARNING_RATE)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 150MB/s]


In [ ]:
for epoch in range(EPOCHS):
    for images in celeba_loader:
        images = images.to(DEVICE)  # Ensure images are on the correct device

        # 1. Encode images with CLIP
        with torch.no_grad():
            clip_features = clip_model.encode_image(images).float().to(DEVICE)

        # 2. Map CLIP features to ProGAN latent space
        latent_vectors = mapping_network(clip_features).to(DEVICE)

        # 3. Generate images using ProGAN
        generated_images = progan.test(latent_vectors).to(DEVICE)  # Latent vectors are mapped directly to ProGAN

        # 4. Compute Reconstruction Loss
        loss_reconstruction = criterion_reconstruction(
            F.interpolate(generated_images, size=(IMAGE_SIZE, IMAGE_SIZE), mode='bilinear', align_corners=False),
            images
        )

        # 5. Compute Perceptual Loss
        with torch.no_grad():
            features_real = perceptual_model(images)
            features_fake = perceptual_model(F.interpolate(generated_images, size=(IMAGE_SIZE, IMAGE_SIZE), mode='bilinear', align_corners=False))
        loss_perceptual = torch.mean((features_real - features_fake) ** 2)
        generated_images_resized = F.interpolate(generated_images, size=(224, 224), mode='bilinear', align_corners=False).to(DEVICE)

        # 6. Compute Adversarial Loss
        real_preds = discriminator(images)
        fake_preds = discriminator(generated_images_resized.detach().detach())
        loss_adv_real = criterion_adversarial(real_preds, torch.ones_like(real_preds))
        loss_adv_fake = criterion_adversarial(fake_preds, torch.zeros_like(fake_preds))
        loss_adversarial = loss_adv_real + loss_adv_fake

        # 7. Compute Identity Loss
        loss_identity = 1 - criterion_identity(clip_features, clip_model.encode_image(F.interpolate(generated_images, size=(IMAGE_SIZE, IMAGE_SIZE), mode='bilinear', align_corners=False)).float())

        # 8. Compute Diversity Loss
        noise = torch.randn_like(clip_features).to(DEVICE)
        perturbed_features = clip_features + noise
        perturbed_features = perturbed_features.to(DEVICE)
        perturbed_latents = mapping_network(perturbed_features)
        diverse_images = progan.test(perturbed_latents).to(DEVICE)
        loss_diversity = diversity_loss(
            clip_features,
            perturbed_features,
            generated_images,
            diverse_images
        )

        # 9. Compute Total Loss
        total_loss = (
            loss_reconstruction +
            0.1 * loss_perceptual +
            0.01 * loss_adversarial +
            0.1 * loss_identity +
            0.1 * loss_diversity
        )
        total_loss = total_loss.mean()
        # Backpropagation
        optimizer.zero_grad()
        discriminator_optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        discriminator_optimizer.step()

    print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {total_loss.item():.4f}")

# Save the trained mapping network
torch.save(mapping_network.state_dict(), "mapping_network_newweights.pth")



Epoch 1/1, Loss: 1.3112


In [ ]:
# prompt: also save the weights of discriminator as we are saving of the mapping network

# Save the trained discriminator
torch.save(discriminator.state_dict(), "discriminator_newweights.pth")

In [ ]:
import torch
from PIL import Image
import clip

# Load CLIP model
clip_model, preprocess = clip.load("ViT-B/32", device=DEVICE)
clip_model.eval()

# Load ProGAN from PyTorch Hub
progan = torch.hub.load('facebookresearch/pytorch_GAN_zoo:hub', 'PGAN', model_name='celebAHQ-256', pretrained=True, useGPU=torch.cuda.is_available())

# Load the trained Mapping Network
class MappingNetwork(nn.Module):
    def __init__(self, input_dim=512, latent_dim=512):
        super(MappingNetwork, self).__init__()
        layers = []
        for _ in range(12):
            layers.append(nn.Linear(input_dim, input_dim))
            layers.append(nn.LeakyReLU(0.2))
        layers.append(nn.Linear(input_dim, latent_dim))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

mapping_network = MappingNetwork().to(DEVICE)
mapping_network.load_state_dict(torch.load("/content/mapping_network_newweights.pth", map_location=DEVICE))
mapping_network.eval()

# Function for Generating Images from Text
def generate_image_from_text(text, output_path="generated_image_new.png"):
    # Encode text using CLIP
    with torch.no_grad():
        text_features = clip_model.encode_text(clip.tokenize([text]).to(DEVICE)).float()

    # Map text features to ProGAN latent space
    with torch.no_grad():
        latent_vector = mapping_network(text_features)

    # Generate image using ProGAN
    with torch.no_grad():
        generated_image = progan.test(latent_vector)

    # Convert image to PIL format and save
    generated_image = (generated_image[0].clamp(-1, 1) + 1) / 2  # Normalize to [0, 1]
    generated_image = (generated_image * 255).permute(1, 2, 0).cpu().numpy().astype('uint8')
    Image.fromarray(generated_image).save(output_path)
    print(f"Image saved at {output_path}")

# Example Usage
text_description = "woman"
generate_image_from_text(text_description)


Using cache found in /root/.cache/torch/hub/facebookresearch_pytorch_GAN_zoo_hub


Average network found !
Image saved at generated_image_new.png


<ipython-input-17-4fa9421a6f38>:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mapping_network.load_state_dict(torch.load("/content/mapping_network_newweights.pth", map_l